# TM10007 Group Assignment Machine Learning
#### Sara Arman, Judith Essenburg, George Franssen, Naomi Verkerk

## Import


In [ ]:

from load_data import load_data

## Loading Data

In [27]:

data = load_data() 

print(f'The number of samples/patients: {len(data.index)}')
print(f'The number of columns/features: {len(data.columns)}')


The number of samples: 167
The number of columns: 725


## Data variables